<a href="https://colab.research.google.com/github/DoubekSeth/CS5350HW/blob/main/DecisionTree/CS5350HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data sets

## Libraries

In [95]:
from collections import Counter
import queue
import math
import statistics
import copy
import numpy as np
import pandas as pd

## Datasets

### Tennis

In [96]:
s_tennis = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"]]
full_attributes_tennis = {"outlook":["sunny", "overcast", "rainy"], "temperature":["hot", "mild", "cool"], "humidity":["high", "normal", "low"], "wind":["strong", "weak"]}
remaining_attributes_tennis = ["outlook", "temperature", "humidity", "wind"]

### Boolean Table Example (1st problem)


In [97]:
s_booltab = [
    ["0", "0", "1", "0", "0"],
    ["0", "1", "0", "0", "0"],
    ["0", "0", "1", "1", "1"],
    ["1", "0", "0", "1", "1"],
    ["0", "1", "1", "0", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "1", "0", "1", "0"]
]
full_attributes_booltab = {"x1":["0", "1"], "x2":["0", "1"], "x3":["0", "1"], "x4":["0", "1"]}
remaining_attributes_booltab = ["x1", "x2", "x3", "x4"]

### Car

In [98]:
s_cartraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/train.csv", header=None).to_numpy()
s_cartesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/test.csv", header=None).to_numpy()

full_attributes_car = {
    "buying":["vhigh", "high", "med", "low"],
    "maint":["vhigh", "high", "med", "low"],
    "doors":["2", "3", "4", "5more"],
    "persons":["2", "4", "more"],
    "lug_boot":["small", "med", "big"],
    "safety":["low", "med", "high"]}
remaining_attributes_car = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]

### Bank

In [99]:
s_banktraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/train.csv", header=None).to_numpy()
s_banktesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/test.csv", header=None).to_numpy()

full_attributes_bank = {
    "age":"numeric",
    "job":["admin.","unknown","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services"],
    "marital":["married","divorced","single"],
    "education":["unknown","secondary","primary","tertiary"],
    "default":["yes", "no"],
    "balance":"numeric",
    "housing":["yes", "no"],
    "loan":["yes", "no"],
    "contact":["unknown","telephone","cellular"],
    "day":"numeric",
    "month":["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "duration":"numeric",
    "campaign":"numeric",
    "pdays":"numeric",
    "previous":"numeric",
    "poutcome":["unknown","other","failure","success"]
}
remaining_attributes_bank = ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome"]

Filling in unknowns in dataset

In [100]:
#Find all modes for each data point
attributes_mode_bank = {}
for i in range(len(remaining_attributes_bank)):
  mode = max(set([k for k in s_banktraining[:, i] if k != "unknown"]), key=[k for k in s_banktraining[:, i] if k != "unknown"].count)
  attributes_mode_bank[remaining_attributes_bank[i]] = mode

# Set the data back
s_banktraining_filled = copy.deepcopy(s_banktraining)
for datapoint in range(len(s_banktraining_filled)):
  for eleInd in range(len(s_banktraining_filled[datapoint])):
    if(s_banktraining_filled[datapoint, eleInd]=="unknown"):
      s_banktraining_filled[datapoint, eleInd]=attributes_mode_bank[remaining_attributes_bank[eleInd]]

# Set the data for testing back
s_banktesting_filled = copy.deepcopy(s_banktesting)
for datapoint in range(len(s_banktesting_filled)):
  for eleInd in range(len(s_banktesting_filled[datapoint])):
    if(s_banktesting_filled[datapoint, eleInd]=="unknown"):
      s_banktesting_filled[datapoint, eleInd]=attributes_mode_bank[remaining_attributes_bank[eleInd]]

full_attributes_bank_filled = {
    "age":"numeric",
    "job":["admin.","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services"],
    "marital":["married","divorced","single"],
    "education":["secondary","primary","tertiary"],
    "default":["yes", "no"],
    "balance":"numeric",
    "housing":["yes", "no"],
    "loan":["yes", "no"],
    "contact":["telephone","cellular"],
    "day":"numeric",
    "month":["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "duration":"numeric",
    "campaign":"numeric",
    "pdays":"numeric",
    "previous":"numeric",
    "poutcome":["other","failure","success"]
}

# ID3 Algorithm

## Main Algorithm

In [101]:
class Node:
  def __init__(self, s, atrs, parent=None, parentVal=None, label=None):
    self.s = s
    self.atrs = atrs
    self.parent = parent
    self.label = label
    self.children = []
    self.parentVal = parentVal

  def addChild(self, node):
    self.children.append(node)

In [102]:
def ID3(s, remaining_attributes, full_attributes, purity_func, parentNode, weights=None, level=0, stopping=math.inf, printing=False):
  if weights is None:
    weights=np.ones(len(s))
  #First, check if all training examples s have same label. First stopping condition
  if(labelSame(s)[0]):
    if(printing):
      print("Same Label", labelSame(s)[1])
    parentNode.label = labelSame(s)[1]
    return
  #Second, check if exhausted attributes and if so return leaf node with most common label. Second stopping condition
  if(len(remaining_attributes) == 0):
    if(printing):
      print("Exhausted Attributes", mostCommonLabel(s, 1, weights)[0][0])
    parentNode.label = mostCommonLabel(s, 1, weights)[0][0]
    return
  #Third, check if have hit maximum stopping depth
  #print("hit ", level, " out of ", stopping)
  if(level >= stopping):
    parentNode.label = mostCommonLabel(s, 1, weights)[0][0]
    return
  level += 1
  #If didn't stop, then we can move onto the main part of the algorithm
  #Create root if we don't have one
  if(parentNode==None):
    parentNode = Node(s=s, atrs=remaining_attributes, parent=None)
  #Find attribute best splits S
  bestAtr = findBestSplit(s, weights, remaining_attributes, full_attributes, purity_func, printing)
  parentNode.label = bestAtr
  if(printing):
    print(bestAtr)
  less_remaining_attributes = remaining_attributes[:]
  #print(less_remaining_attributes, bestAtr)
  less_remaining_attributes.remove(bestAtr)
  #Go thru all values for best attribute
  loop_attrs = full_attributes[bestAtr]
  numericFilter=False
  if(loop_attrs=="numeric"):
    numericFilter=True
    loop_attrs = ["lower than median", "higher than median"]
  for val in loop_attrs:
    s_v, weights_v = filterSOnAtrVal(s, weights, bestAtr, val, full_attributes, numericFilter)
    #Check if s_v empty
    if(len(s_v)==0):
      if(printing):
        print("S_v empty", val, mostCommonLabel(s, 1, weights)[0][0])
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val, label=mostCommonLabel(s, 1, weights)[0][0])
      parentNode.addChild(newNode)
    else:
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val)
      if(printing):
        print("New node", val)
      parentNode.addChild(newNode)
      ID3(s_v, less_remaining_attributes, full_attributes, purity_func, newNode, weights=weights_v, level=level, stopping=stopping, printing=printing)
  return parentNode

## Helper Functions

Does two functions, first returns true if all labels are the same for s. Second returns the label if all labels are same or else returns none

In [103]:
def labelSame(s):
  label = s[0][len(s[0])-1]
  for i in s:
    if(i[len(i)-1] != label):
      return False, None
  return True, label

Another helper function, returns the n most common label in s

In [104]:
def mostCommonLabel(s, n, weights):
  labels = {}
  for i in range(len(s)):
    if(s[i][len(s[i])-1] in labels):
      labels[s[i][len(s[i])-1]] += weights[i]
    else:
      labels[s[i][len(s[i])-1]] = weights[i]
  #Find most common label out of all the labels
  #print("Labels:", labels)
  mostCommon = sorted(labels.items(), key=lambda x:x[1], reverse=True)
  #print("Most Common:", mostCommon)
  return mostCommon

This helper finds the attribute that results in the best split

In [105]:
def findBestSplit(s, weights, atrs, full_attributes, purity_func, printing):
  max_info_gain = 0
  max_info_gain_atr = atrs[0]
  for atr in atrs:
    info_gained = find_info_gained(s, weights, atr, full_attributes, purity_func, printing)
    if(printing):
      print(atr, info_gained)
    if(info_gained > max_info_gain):
      max_info_gain = info_gained
      max_info_gain_atr = atr
  return max_info_gain_atr

This helper gives the information gained from a single attribute

In [106]:
def find_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  if(full_attributes[atr]=="numeric"):
    return find_numeric_info_gained(s, weights, atr, full_attributes, purity_func, printing)
  else:
    return find_categorical_info_gained(s, weights, atr, full_attributes, purity_func, printing)

In [107]:
def find_numeric_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {"low":[], "high":[]}
  count = 0
  median = statistics.median(x[atr_ind] for x in s)
  for i in s:
    if i[atr_ind] < median:
      atr_type_count_dict["low"].append([i[len(i)-1], weights[count]])
    else:
      atr_type_count_dict["high"].append([i[len(i)-1], weights[count]])
    count += 1
  #This puts s into format of (label, weight)
  pur_s = []
  count = 0
  for i in s:
    pur_s.append([i[len(i)-1], weights[count]])
    count += 1
  starting_info = purity_func(pur_s)
  if(printing):
    print("Starting Info: ", starting_info)
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    weighted_sum += sum(i[1] for i in j)/sum(weights)*purity_func(j)
  return starting_info-weighted_sum

In [122]:
def find_categorical_info_gained(s, weights, atr, full_attributes, purity_func, printing):
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {}
  count = 0
  for i in s:
    if i[atr_ind] in atr_type_count_dict:
      atr_type_count_dict[i[atr_ind]].append([i[len(i)-1], weights[count]])
    else:
      atr_type_count_dict[i[atr_ind]] = [[i[len(i)-1], weights[count]]]
    count += 1
  #Use proportions to run info gain function and find total information gain
  #This puts s into format of (label, weight)
  pur_s = []
  count = 0
  for i in s:
    pur_s.append([i[len(i)-1], weights[count]])
    count += 1
  starting_info = purity_func(pur_s)
  if(printing):
    print("Starting Info: ", starting_info)
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    if(printing):
      print("J", j)
    weighted_sum += sum(i[1] for i in j)/sum(weights)*purity_func(j)
  return starting_info-weighted_sum

Filters S based on an attribute value to get S_v

In [109]:
def filterSOnAtrVal(s, weights, atr, val, full_attributes, numericFilter):
  filtered = []
  filteredWeights = []
  count = 0
  atr_ind = list(full_attributes.keys()).index(atr)
  for i in s:
    if(numericFilter):
      if(val == "lower than median"):
        if(i[atr_ind] < statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
          filteredWeights.append(weights[count])
      else:
        if(i[atr_ind] >= statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
          filteredWeights.append(weights[count])
    else:
      if(i[atr_ind] == val):
        filtered.append(i)
        filteredWeights.append(weights[count])
    count += 1
  return filtered, filteredWeights

## Purity Functions

In [110]:
def entropy(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  #print(grouped)
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  return -np.dot(vec_props, np.log2(vec_props))

In [111]:
def GiniInd(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  return 1-np.dot(vec_props, vec_props)

In [112]:
def majorityError(labels):
  labeldf = pd.DataFrame(labels, columns=["Label", "Weight"])
  grouped = labeldf.groupby("Label").sum()
  vec_counts = np.array(grouped).flatten()
  vec_props = vec_counts/np.sum(vec_counts)
  vec_props.sort()
  return np.sum(vec_props[:len(vec_props)-1])

## Display Tree

In [113]:
def unravelTree(q1, q2, childQ, level):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  while(not q1.empty()):
    node = q1.get()
    childThere = childQ.get()
    firstChild = True
    print("ParentVal: ", node.parentVal)
    print("Label: ", node.label)
    if(not childThere):
      print("----")
    for child in node.children:
      q2.put(child)
      if(firstChild):
        firstChild = False
      else:
        childQ.put(True)
    if(len(node.children)!= 0):
      childQ.put(False)
  level += 1
  q1 = q2
  q2 = queue.Queue()
  if(q1.empty()):
    return
  unravelTree(q1, q2, childQ, level)

This one creates a tree and returns the root node

In [114]:
def createTree(s, rem, full_attr, pur, weights, stop, printing=False):
    root = ID3(s, rem, full_attr, pur, None, weights=weights, stopping=stop, printing=printing)
    return root

This prints a tree

In [115]:
def displayTree(root):
    q = queue.Queue()
    q2 = queue.Queue()
    q.put(root)
    childQ = queue.Queue()
    childQ.put(True)

    unravelTree(q, q2, childQ, 0)

# Run ID3

## Set Data

index = 3
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 1

## Display Tree

weights = np.ones(len(s))
print(weights)
root = ID3(s, rem, full_attr, pur, None, weights=weights, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

## Evaluate Model

Finds accuracy of model on test data

In [118]:
def findAcc(root, test, full_attr):
  matched = 0
  for i in test:
    matched += travTree(root, i, full_attr)
  return matched/len(test)

Traverses the tree recursively, returns 1 if test label matches model and 0 otherwise

In [119]:
def travTree(root, datapoint, full_attr):
  #Stopping Condition, root no children
  if(len(root.children) == 0):
    if(root.label == datapoint[len(datapoint)-1]):
      #print("matched")
      return 1
    else:
      #print("no match")
      return 0
  #Otherwise, traverse down
  curr_attr = root.label
  do_numeric = False
  if(full_attr[curr_attr] == "numeric"):
    do_numeric = True
  #print("current attribute: ", curr_attr)
  atr_ind = list(full_attr.keys()).index(curr_attr)
  for child in root.children:
    if(do_numeric):
      median = statistics.median(x[atr_ind] for x in root.s)
      #print("numeric: ", median, " ", datapoint[atr_ind])
      if((datapoint[atr_ind] < median and child.parentVal=="lower than median") or (datapoint[atr_ind] >= median and child.parentVal=="higher than median")):
        return travTree(child, datapoint, full_attr)
    else:
      if(child.parentVal == datapoint[atr_ind]):
        #print("Going down on val: ", child.parentVal)
        return travTree(child, datapoint, full_attr)

root = ID3(s, rem, full_attr, pur, None, stopping=stop)

findAcc(root, s_test, full_attr)

# Run Boolean Table

## Set Vars

index = 0
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = math.inf

## Run ID3 & Print

root = ID3(s, rem, full_attr, pur, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

# Run Tennis Example

## Set Vars

index = 1
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = math.inf

## Run ID3

### Entropy

root = ID3(s, rem, full_attr, entropy, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

### Gini Index

root = ID3(s, rem, full_attr, GiniInd, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

### Majority Error

root = ID3(s, rem, full_attr, majorityError, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

#Using modified Tennis Example

In [120]:
s_tennis_modified = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"],
     #Modifications buddy
     ["overcast", "mild", "normal", "weak", "+"]]

root = ID3(s_tennis_modified, rem, full_attr, entropy, None, stopping=stop, printing=True)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

# Used to help with fractional counts

def entropyWeighted(positive, negative):
  whole = positive + negative
  return (-(positive/whole)*math.log2(positive/whole) if positive/whole > 0 else 0)-((negative/whole)*math.log2(negative/whole) if negative/whole > 0 else 0)

entropyWeighted(3+5/14, 2)

poss = [3+5/14, 2+5/14]
negs = [1, 1]
S = 15
summ = 0
for i in range(len(poss)):
  S_v = poss[i] + negs[i]
  print("Pos: ", poss[i], " Neg: ", negs[i], " entropy weighted: ", (S_v)/S*entropyWeighted(poss[i], negs[i]))
  summ += (S_v)/S*entropyWeighted(poss[i], negs[i])
summ

# Car Example problem 2

## Set Data

can set stop to halt tree growth after a certain number of steps

index = 2
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

## Running with entropy

root = ID3(s, rem, full_attr, entropy, None, stopping=stop)

findAcc(root, s_test, full_attr)

## Running with gini index

root = ID3(s, rem, full_attr, GiniInd, None, stopping=stop)

findAcc(root, s_test, full_attr)

## Running with majority error

root = ID3(s, rem, full_attr, majorityError, None, stopping=stop)

findAcc(root, s_test, full_attr)

## Running through all levels & purity functions

for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 7):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)

# Bank Example Problem 3

## Unknowns as a value

index = 3
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktraining_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 17):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)
    

## Car with unknowns filled

index = 4
s = [s_booltab, s_tennis, s_cartraining, s_banktraining, s_banktraining_filled][index]
s_test = ["","", s_cartesting, s_banktesting, s_banktesting_filled][index]
rem = [remaining_attributes_booltab, remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank, remaining_attributes_bank][index]
full_attr = [full_attributes_booltab, full_attributes_tennis, full_attributes_car, full_attributes_bank, full_attributes_bank_filled][index]
pur = [entropy, GiniInd, majorityError][0]
stop = 4

for purity in range(0, 3):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Using:", ["entropy", "GiniInd", "majorityError"][purity])
  for i in range(1, 17):
    root = ID3(s, rem, full_attr, [entropy, GiniInd, majorityError][purity], None, stopping=i)
    accTest = 1-findAcc(root, s_test, full_attr)
    accTrain = 1-findAcc(root, s, full_attr)
    print("Stopping at", i, "obtains test error", accTest, "and training error", accTrain)